In [3]:
import jupytools.syspath
jupytools.syspath.add('..')

import csv
import json

import pandas as pd
from tqdm.notebook import tqdm

from basedir import TRAIN, TRAIN_LABELS, TRAIN_SPECS

In [4]:
trn_data = pd.read_csv(TRAIN)
trn_target = pd.read_csv(TRAIN_LABELS)
trn_specs = pd.read_csv(TRAIN_SPECS)

In [6]:
trn_data.head(2)

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK


In [7]:
trn_target.head(2)

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0


In [8]:
trn_specs.head(3)

,event_id,info,args
0,2b9272f4,The end of system-initiated feedback (Correct)...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
1,df4fe8b6,The end of system-initiated feedback (Incorrec...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
2,3babcb9b,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."


In [14]:
trn_data.shape

(11341042, 11)

In [16]:
trn_dataset = pd.merge(
    pd.merge(trn_data, 
             trn_target, 
             on=['game_session', 'installation_id'], 
             how='left'),
    trn_specs,
    on='event_id',
    how='left'
)

In [17]:
trn_dataset.sample(3)

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title_x,type,world,title_y,num_correct,num_incorrect,accuracy,accuracy_group,info,args
8408239,cfbd47c8,a6d69813b3dc8781,2019-07-29T01:30:04.809Z,"{""weight"":1,""source"":""resources"",""resources"":[...",bbb84efa,52,4030,59641,Chow Time,Game,CRYSTALCAVES,NaN,NaN,NaN,NaN,NaN,This event occurs when the player clicks to pi...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
3718667,67439901,2326638c3784a1b6,2019-09-23T16:32:44.654Z,"{""description"":""I'll add this one to your coll...",52518080,77,3010,87346,Bottle Filler (Activity),Activity,MAGMAPEAK,NaN,NaN,NaN,NaN,NaN,The system-initiated instruction event occurs ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
6142804,71fe8f75,7729edbee3e1678c,2019-10-03T01:37:20.658Z,"{""description"":""Drag a bug over here! idle"",""i...",8833c392,7,3110,16822,Bug Measurer (Activity),Activity,TREETOPCITY,NaN,NaN,NaN,NaN,NaN,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."


In [2]:
def flatten(x, acc=None, prefix=''):
    def _parse(k, v, a):
        if isinstance(v, dict):
            for new_k, new_v in v.items():
                _parse(f'{k}_{new_k}', new_v, a)
        elif isinstance(v, list):
            for i, new_v in enumerate(v):
                _parse(f'{k}_{i}', new_v, a)
        elif isinstance(v, (int, float, str)):
            acc[k] = v
    
    acc = {}
    for key, value in x.items():
        _parse(key, value, acc)
    return acc

In [25]:
n_lines = 11341042
rows = []
with TRAIN.open() as lines:
    reader = csv.DictReader(lines)
    for line in tqdm(reader, total=n_lines):
        obj = json.loads(line['event_data'])
        obj = flatten(obj)
        rows.append(obj)

In [32]:
from itertools import chain
keys = set(chain(*[list(r.keys()) for r in rows]))

In [37]:
from collections import defaultdict
groups = defaultdict(list)
for row in rows:
    code = row['event_code']
    groups[code].append(row)

In [40]:
groups.keys()

dict_keys([2000, 3010, 3110, 4070, 4090, 4030, 4035, 4021, 4020, 4010, 2080, 2083, 2040, 2020, 2030, 3021, 3121, 2050, 3020, 3120, 2060, 2070, 4031, 4025, 5000, 5010, 2081, 2025, 4022, 2035, 4040, 4100, 2010, 4110, 4045, 4095, 4220, 2075, 4230, 4235, 4080, 4050])

In [97]:
import random
import numpy as np
from pprint import pprint
from IPython.display import display, HTML
for code, examples in groups.items():
    tag = f'<strong>Code: {code}</strong>'
    if examples:
        sample = pd.DataFrame(np.random.choice(examples, size=5).tolist())
        tag += sample.to_html()
    else:
        tag += '<strong>No events</strong>'
    display(HTML(tag))

,event_code,event_count,version,round,game_time
0,2000,1,NaN,NaN,NaN
1,2000,1,1,0.0,0.0
2,2000,1,1.0,0.0,0.0
3,2000,1,NaN,NaN,NaN
4,2000,1,1.0,NaN,0.0


,description,identifier,media_type,total_duration,level,round,event_count,game_time,event_code
0,Can you match the animals to the pails of water that are just the right size?,water_question,animation,4900,2.0,3,30,41824,3010
1,That looks so cool!,thatLooksSoCool,animation,2533,NaN,16,216,435420,3010
2,Which soap is the right size for the cow?,cow_whichsoap,animation,2866,6.0,14,150,133381,3010
3,Which tub is right for the hog?,hog_whichtub,animation,2366,7.0,16,230,386511,3010
4,This is how much water one container holds.,Mom_ThatsHowMuchWater,audio,3360,NaN,2,167,123038,3010


,description,identifier,media_type,duration,event_count,game_time,event_code,round
0,So cool!,Dot_SoCool,audio,1225,558,574057,3110,NaN
1,We've got to balance this scale! Fill the trays with gems so both sides weigh the same! Use your finger to drag the gems to the trays!,"Dot_BalanceScale,Dot_FillTrays,Dot_UseFingerDragGems",audio,7053,5,7094,3110,NaN
2,Five!,sid_5,animation,650,29,33446,3110,NaN
3,Try adding eggs to a different nest.,Buddy_TryDifferentNest,audio,2726,82,153526,3110,NaN
4,"And it's full! I'll put this one over here, so you can make another one!",andItsFull,animation,4473,289,581415,3110,25.0


,coordinates_x,coordinates_y,coordinates_stage_width,coordinates_stage_height,level,round,event_count,game_time,event_code
0,503,96,1014,541,7.0,18,249,213032,4070
1,311,442,1015,762,NaN,2,725,284322,4070
2,845,332,1014,468,NaN,0,75,43297,4070
3,853,5,1015,406,NaN,0,51,98266,4070
4,220,748,1015,762,NaN,2,112,99679,4070


,flowers_0,flowers_1,flowers_2,flowers_3,flowers_4,event_count,game_time,event_code,target_water_level,water_level,round,hats_0,hats_1,hats_2,caterpillars_0,caterpillars_1,caterpillars_2,mode,scale_weights,target_weight,table_weights
0,0.0,0.0,0.0,0.0,0.0,105,144103,4090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,0.0,0.0,0.0,0.0,467,686235,4090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,29,45712,4090,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,16,11787,4090,NaN,NaN,NaN,0.0,0.0,0.0,2.0,2.0,2.0,caterpillars,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,125,186338,4090,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,5.0


...

,duration,tutorial_step,round,event_count,game_time,event_code
0,5300,1,0,7,9604,2075
1,3967,1,0,8,12061,2075
2,6918,1,0,8,12043,2075
3,2083,1,3,48,73507,2075
4,10052,3,0,15,19289,2075


,toy,coordinates_x,coordinates_y,coordinates_stage_width,coordinates_stage_height,round,event_count,game_time,event_code
0,ship,467,382,1014,541,2,166,140979,4230
1,ball,495,380,1014,541,1,68,80267,4230
2,ball,620,189,1014,541,1,70,74888,4230
3,ufo,538,308,1014,541,1,90,89752,4230
4,ball,373,428,1014,541,1,67,116191,4230


,toy,coordinates_x,coordinates_y,coordinates_stage_width,coordinates_stage_height,round,event_count,game_time,event_code
0,ufo,253,400,1014,541,1,40,76350,4235
1,rocket,494,445,1014,541,6,965,538226,4235
2,duck,600,137,1014,541,4,341,315042,4235
3,ufo,733,392,1014,541,7,381,458587,4235
4,duck,428,457,1014,541,3,183,186306,4235


,dwell_time,object,round,event_count,game_time,event_code
0,1084,petey,8.0,244,334305,4080
1,2148,window,1.0,38,89954,4080
2,5924,bottle_2,1.0,12,14209,4080
3,1670,flower_2,NaN,196,416768,4080
4,4151,grassHopper,NaN,281,207388,4080


,max_position,end_position,duration,round,event_count,game_time,event_code
0,1.000000,1.00000,1591,2,40,56337,4050
1,1.000000,1.00000,8609,4,88,125683,4050
2,0.586982,0.55858,1334,1,14,28535,4050
3,0.000000,0.00000,205,1,35,25011,4050
4,1.000000,1.00000,2641,1,45,56291,4050


In [43]:
trn_df = pd.read_csv(TRAIN)

In [44]:
trn_df.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,45bb1e1b6b50c07b,2019-09-06T17:53:46.937Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,17eeb7f223665f53,2019-09-06T17:54:17.519Z,"{""event_code"": 2000, ""event_count"": 1}",0001e90f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,77261ab5,0848ef14a8dc6892,2019-09-06T17:54:56.302Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0001e90f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
3,b2dba42b,0848ef14a8dc6892,2019-09-06T17:54:56.387Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,2,3010,53,Sandcastle Builder (Activity),Activity,MAGMAPEAK
4,1bb5fbdb,0848ef14a8dc6892,2019-09-06T17:55:03.253Z,"{""description"":""Let's build a sandcastle! Firs...",0001e90f,3,3110,6972,Sandcastle Builder (Activity),Activity,MAGMAPEAK


In [45]:
trn_df.shape

(11341042, 11)

In [51]:
trn_labels_df = pd.read_csv(TRAIN_LABELS)

In [52]:
trn_labels_df.head()

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3


In [53]:
trn_specs_df = pd.read_csv(TRAIN_SPECS)

In [54]:
trn_specs_df.head()

,event_id,info,args
0,2b9272f4,The end of system-initiated feedback (Correct)...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
1,df4fe8b6,The end of system-initiated feedback (Incorrec...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
2,3babcb9b,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
3,7f0836bf,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."
4,ab3136ba,The end of system-initiated instruction event ...,"[{""name"":""game_time"",""type"":""int"",""info"":""mill..."


In [55]:
trn_df.shape, trn_labels_df.shape, trn_specs_df.shape

((11341042, 11), (17690, 7), (386, 3))

In [ ]:
trn_specs_df

In [73]:
trn_df['description'] = trn_df.event_id.map(trn_specs_df.set_index('event_id')['info'].to_dict())

In [79]:
codes = (trn_df
         .drop_duplicates(subset=['description'])[['event_code', 'description']]
         .reset_index(drop=True))

In [92]:
from IPython.display import display, HTML
content = [
    f'<caption><strong>Total number of events: {len(codes)}</strong></caption>'
    '<thead><th>Code</th><th>Description</th></thead>'
]
for r in codes.sort_values(by='event_code').to_dict('record'):
    code, descr = r['event_code'], r['description']
    content.append(f'<tr><td>{code}</td><td>{descr}</td></tr>')
HTML(f"<table>{''.join(content)}</table>")

2000,This event is triggered when a video starts playing
2000,"The start game event is triggered at the very beginning of the level (after the game finishes loading, don't wait for intro movie to finish). This is used to compute things like time spent in game."
2000,"The start game event is triggered at the very beginning of the game (after the game finishes loading, don't wait for intro movie to finish). It includes information on the current version of the game loaded. This is used to compute things like time spent in game."
2000,"The start game event is triggered at the very beginning of the game (after the game finishes loading, don’t wait for intro movie to finish). This is used to compute things like time spent in game."
2000,The start game event is triggered at the very beginning of the level (after the game finishes loading don't wait for intro movie to finish). This is used to compute things like time spent in game.
2010,The exit game event is triggered when the game is quit. This is used to compute things like time spent in game. Depending on platform this may / may not be possible. NOTE: “quit” also means navigating away from game.
2010,"The exit game event is triggered when the game is quit. This is used to compute things like time spent in game. Depending on platform this may / may not be possible. NOTE: ""quit"" also means navigating away from game."
2020,The start round event is triggered at the start of a round when the player is prompted to fill a jar. It is used in calculating things like time spent in a round and the number of rounds the player has visited (exposures).
2020,"The start round event is triggered at the start of a round when the player is prompted to weigh and arrange the chests. There is only one round per playthrough. This event provides information about the game characteristics of the round (i.e. resources, objectives, setup). It is used in calculating things like time spent in a round (for speed and accuracy), attempts at solving a round, and the number of rounds the player has visited (exposures)."
2020,The start round event is triggered at the start of the game. It contains information about the layout of the activity.
2020,"The start round event is triggered at the start of a round when a new arrangement of dinosaurs and houses is presented. This event provides information about the game characteristics of the round (i.e. resources, objectives, setup). It is used in calculating things like time spent in a round (for speed and accuracy), attempts at solving a round, and the number of rounds the player has visited (exposures)."
